In [72]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np
import spacy

ModuleNotFoundError: No module named 'spacy'

## Chargement des données

In [2]:
train = pd.read_csv('BUILD/train.csv')
dev = pd.read_csv('BUILD/dev.csv')

## Extraction des descripteurs

Le ColumnTransformer permet de calculer des descripteurs simplement à partir d'un dataframe.
Il s'agit d'une liste de 3 élements:
    - le nom du descripteur (str)
    - une fonction ou un objet scikit-learn qui implémente `fit` et ou `transform`
    - la colonne à utiliser

Par exemple pour le descripteur `sentence_position`, on va utiliser la colonne `sentence_percent` directement (`passthrough`) (il faut utiliser des crochets pour que la taille du vecteur soit correcte).

In [3]:
# Cette fonction reçoit les colonnes choisies pour tout les exemples
# et renvoie un array numpy avec de taille (nb_exemple, nb_features)
def sentence_position(df):
    # print(df.shape, df.columns)
    
    # calcule le nombre de phrase par document dans un dictionnaire
    doc2nbsent = df.groupby('doc_id')['sentence_index'].max().to_dict()
    # crée une colonne qui associe à chaque document_id le nombre de phrase du document
    df['nb_sent'] = df['doc_id'].map(doc2nbsent.get)
    # calcul le rapport entre l'indice de la phrase et le nombre total de phrases
    position = df['sentence_index'] / df['nb_sent']

    # Converti la colonne pandas (pd.Series) en array numpy (np.ndarray) puis lui
    # donne la taille (n_samples, n_features_new), -1 représente le nombre d'élément
    # original et 1 la nouvelle feature que l'on a créée
    return position.values.reshape(-1, 1)

In [4]:
def contains_ldots(sentence):
    return 1 if '...' in sentence else 0

In [5]:
dev['contains_ldots'] = dev['text'].map(contains_ldots)
train['contains_ldots'] = train['text'].map(contains_ldots)

Ajout du descripteur Sentence Length:

In [20]:
# Fonction pour extraire la longueur de chaque phrase en nombre de mots
def sentence_length(df):
    # Utilisez la fonction str.split() pour diviser chaque phrase en mots et compter leur nombre
    lengths = df['text'].apply(lambda x: len(x.split()))
    # Convertissez la colonne pandas en array numpy et redimensionnez-la
    return lengths.values.reshape(-1, 1)


In [60]:
# Il y a plusieurs manières de combiner les descripteurs:
# - utiliser des objets de SkLearn (comme le CountVectorizer)
# - utiliser un FunctionTransformer qui prend en entrée une fonction qui va créer le descripteur à la volée
# - utiliser le mot-clé "passthrough" pour les descripteurs précaculés
column_trans = ColumnTransformer(
    [
        (
            "count_vectorizer",
            CountVectorizer(stop_words='english',ngram_range=(1,3), min_df=10),
            "text",
        ),
        (
            "single_values",
            'passthrough', # passthrough permet de passer les colonnes sans les transformer
            ["contains_ldots"],  # ajoutez d'autre noms de colonnes
        ),
        (
            'sentence_position', # nom du descripteur
            FunctionTransformer(sentence_position),
            ['doc_id', 'sentence_index'],  # les colonnes à transmettre à la fonction sentence_position
        ),
        (
            "Tfidf Vectorizer",
            TfidfVectorizer(stop_words='english', ngram_range=(1, 3), min_df=10),
            "text",
        ),
        #(
        #    'Sentence Length', # descripteur de longueur de phrase 
        #    FunctionTransformer(sentence_length),
        #    ['text'],  # Colonnes de texte pour lesquelles vous voulez calculer la longueur
        #),

    ],
    remainder="drop",
)

# Ici on crée un pipeline qui va extraire les descripteurs puis entraîner ou prédire
classifier = make_pipeline(
    column_trans,
    LogisticRegression(multi_class="multinomial")
)

## Entraînement

In [61]:
# On entraîne le modèle sur les 100 premiers exemples.
classifier.fit(train, train['labels'])

C:\Users\hugol\AppData\Roaming\Python\Python38\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('count_vectorizer',
                                                  CountVectorizer(min_df=10,
                                                                  ngram_range=(1,
                                                                               3),
                                                                  stop_words='english'),
                                                  'text'),
                                                 ('single_values',
                                                  'passthrough',
                                                  ['contains_ldots']),
                                                 ('sentence_position',
                                                  FunctionTransformer(func=<function sentence_position at 0x0000019A8C6F40D0>),
                                                  ['doc_id', 'sentence_index']),
                                                 ('Tfidf Vectorizer',
                                                  TfidfVectorizer(min_df=10,
                                                                  ngram_range=(1,
                                                                               3),
                                                                  stop_words='english'),
                                                  'text')])),
                ('logisticregression',
                 LogisticRegression(multi_class='multinomial'))])

## Prédiction

In [62]:
# Prédit puis choisis l'étiquette qui a la plus haut probabilité
dev['pred'] = classifier.predict(dev)
train['pred'] = classifier.predict(train)

In [63]:
# Pour obtenir le nom des étiquettes prédites
# On récupère le nom et l'ordre des étiquettes
index2label = classifier.named_steps['logisticregression'].classes_


# Retourne la probabilité pour chacune des étiquettes
predictions_prob = classifier.predict_proba(dev.iloc[:10])
# On choisis l'étiquette avec la plus haute probabilité
index_max_predictions = predictions_prob.argmax(axis=1).tolist()

# Ces deux liste devraient être les mêmes
print([index2label[l] for l in index_max_predictions])
print(classifier.predict(dev.iloc[:10]).tolist())

['ANALYSIS', 'ANALYSIS', 'RPC', 'RPC', 'RPC', 'NONE', 'PREAMBLE', 'FAC', 'NONE', 'ANALYSIS']
['ANALYSIS', 'ANALYSIS', 'RPC', 'RPC', 'RPC', 'NONE', 'PREAMBLE', 'FAC', 'NONE', 'ANALYSIS']


## Evaluation

In [64]:
from sklearn.metrics import precision_recall_fscore_support

### Erreur d'entraînement

In [65]:
ground_truth_labels = train['labels']
submission_labels = train['pred']

In [66]:
precision, recall, f1, _ = precision_recall_fscore_support(
    ground_truth_labels, submission_labels, average='weighted'
)

In [67]:
# https://pyformat.info/#number
print(f'{precision:.2f}, {recall:.2f}, {f1:.2f}')

0.93, 0.92, 0.92


### Erreur de validation

In [69]:
ground_truth_labels = dev['labels']
submission_labels = dev['pred']

In [70]:
precision, recall, f1, _ = precision_recall_fscore_support(
    ground_truth_labels, submission_labels, average='weighted'
)

In [71]:
# https://pyformat.info/#number
print(f'{precision:.2f}, {recall:.2f}, {f1:.2f}')

0.64, 0.64, 0.62


In [68]:
# Comment évoluent les scores lorsqu'on enlève un descripteur ? lorsqu'on change les paramètres du CountVectorizer ?
# Score avec les descripteurs fournies 
# Score erreur d'entrainement : 0.91, 0.91, 0.91
# Score erreur de validation : 0.63, 0.64, 0.62

# Score en retirant sentence_position
# Score erreur d'entrainement : 0.84, 0.83, 0.83
# Score erreur de validation : 0.54, 0.54, 0.52

# lorsqu'on ajoute un TfIdfVectorizer ?
# Score en ajoutant un TfIdfVectorizer
# Score erreur d'entrainement : 0.93, 0.92, 0.92
# Score erreur de validation : 0.64, 0.64, 0.62

### Matrice de confusion

In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html
# TODO